In [5]:
import arcpy, os
import arcpy
from osgeo import gdal

arcpy.env.addOutputsToMap = False   # Prevents many small grid files to show on map
arcpy.env.overwriteOutput = True

In [6]:
# Directory option 1: Use project gdb
# Directory option 2: use project directory
# Directory option 3: Custom directory
Dir = 2  # [1, 2 or 3] (Default is 1)

if Dir == 1:
    p = arcpy.mp.ArcGISProject("CURRENT")
    directory = p.defaultGeodatabase   
elif Dir == 2:
    directory = os.path.dirname( arcpy.mp.ArcGISProject("CURRENT").filePath )
elif Dir == 3:
    directory = r"H:\DATA\Service\Wietse\GIS\DEM"

arcpy.env.workspace = directory

# Inladen bestanden

In [7]:
Input_Arcpy_Eenmalig = r"H:\Team_Kennis_OSA\Hydraulische toetsing\Basisbestanden Zomer\Input_ArpPy\Eenmalige_bestanden"
Bodemkaart_HDSR = os.path.join(Input_Arcpy_Eenmalig,"Bodemkaart_HDSR.shp")

# Bepalen boden voor stroomsnelheid

In [9]:
arcpy.AddField_management(Bodemkaart_HDSR, "vel_crit", "DOUBLE")

def calculate_value(field_value):
    if "Stedelijk" in field_value or "stedelijk" in field_value:
        return 0.3
    elif "Zand" in field_value or "zand" in field_value:
        return 0.3
    elif "Veen" in field_value or "veen" in field_value:
        return 0.3001
    elif "Klei" in field_value or "klei" in field_value:
        return 0.6
    elif "Zavel" in field_value or "zavel" in field_value:
        return 0.6
    elif "Water" in field_value or "water" in field_value:
        return 0.3
    elif "Moeras" in field_value or "moeras" in field_value:
        return 0.3001
    elif "Overig" in field_value or "overig" in field_value:
        return 0.3001
    else:
        return 0.9

# Calculate the values of the calculation_field using the calculate_value function
with arcpy.da.UpdateCursor(Bodemkaart_HDSR, ["GRONDSOORT", "vel_crit"]) as cursor:
    for row in cursor:
        row[1] = calculate_value(row[0])
        cursor.updateRow(row)


In [ ]:
def perform_spatial_join(input_features, join_features, output_features):
    field_mappings = arcpy.FieldMappings()
    field_mappings.addTable(input_features)
    field_mappings.addTable(join_features)

    output_fields = ["ID", "hydro_segm", "pg_code", "Value", "dir_et", "LENGTE", "vel_crit"]

    for field in field_mappings.fields:
        if field.name not in output_fields:
            field_mappings.removeFieldMap(field_mappings.findFieldMapIndex(field.name))

    arcpy.SpatialJoin_analysis(
        target_features=input_features,
        join_features=join_features,
        out_feature_class=output_features,
        join_operation="JOIN_ONE_TO_ONE",
        join_type="KEEP_ALL",
        field_mapping=field_mappings,
        match_option="INTERSECT",
        search_radius="0 Meters"
    )

# Example usage:
input_features = "SOBEK_ReachSegments_Join_PG_Hydro_Line_20210819_Discharge_Length"
output_features = "SOBEK_ReachSegments_Join_PG_Hydro_Line_20210819_Discharge_Length_Soil_shp"

perform_spatial_join(input_features, Bodemkaart_HDSR, output_features)


# Drooglegging bepalen

# Droogleggingseis toetsen per punt

# Exporteer Sobek resultaten naar kunstwerk punten